In [6]:
import pandas as pd 
import geopandas as gpd
from shapely.geometry import Point
from bbox import * 
from pytz import timezone
import os
from shapely.wkt import loads
import numpy as np
import xarray as xr
import gc

In [ ]:
ds = xr.open_dataset(f'../../data/sensor_data/full_gridded_sensoronly.nc')

# Sensor Data Only

In [2]:
sensor_df=pd.read_csv('../../data/sensor_data/hourly_raw_am.csv', index_col=0)
sensor_df=sensor_df[['latitude', 'longitude', 'date_local',
       'time_local', 'sample_measurement', 'site_number', 'county','state']]
sensor_df=sensor_df[~sensor_df['sample_measurement'].isna()]
sensor_df['time'] = pd.to_datetime(sensor_df['date_local'] + ' ' + sensor_df['time_local'])


no2_hours = pd.read_csv("../../data/tempo_data/no2_file_hours.csv")
no2_hours=list(no2_hours['time_hr_ct'])
time_list = np.array(no2_hours, dtype="datetime64[ns]")
sensor_time_list = np.array(sensor_df['time'].unique(), dtype="datetime64[ns]")
seta = set(sensor_time_list)
setb = set(time_list)
union_set = seta & setb
full_time_list = list(union_set)
# Convert to pandas datetime objects
pd_datetime_list = list(pd.to_datetime(full_time_list))
sensor_df['time']=pd.to_datetime(sensor_df['time'])
sensor_df=sensor_df[sensor_df['time'].isin(pd_datetime_list)]

In [3]:
# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(
    sensor_df,
    geometry=[Point(xy) for xy in zip(sensor_df["longitude"], sensor_df["latitude"])],
    crs="EPSG:4326"  # WGS84 coordinate reference system
)
gdf=gdf[['latitude','longitude','geometry']].drop_duplicates()

# Create gridded GDF
res = 0.01 
lon_bins = np.arange(lon_min+0.005, lon_max, res)
lat_bins = np.arange(lat_min+0.005, lat_max, res)

grid_cells = []
for x in lon_bins[:-1]:
    for y in lat_bins[:-1]:
        grid_cells.append(
            Point(x, y).buffer(res / 2).envelope
        )

grid_gdf = gpd.GeoDataFrame(
    {"geometry": grid_cells},
    crs="EPSG:4326"
)
grid_gdf["lat"] = grid_gdf.geometry.centroid.y
grid_gdf["lon"] = grid_gdf.geometry.centroid.x

key = gpd.sjoin(grid_gdf, gdf, how="inner", predicate="intersects")
key=key[['geometry','lat','lon','latitude','longitude']]

sensor_df_joined = pd.merge(sensor_df, key, on=['latitude','longitude'], how='left')
# sensor_df_joined = sensor_df_joined[['lat', 'lon', 'time', 'sample_measurement']]
sensor_df_joined=sensor_df_joined.rename(columns={'sample_measurement':'no2'})

del gdf
del sensor_df
del grid_cells
del key
gc.collect()

sensor_df_joined.to_csv('../../data/sensor_data/final_sensor_withgrid.csv')

/var/tmp/ipykernel_27677/1432961727.py:25: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  grid_gdf["lat"] = grid_gdf.geometry.centroid.y
/var/tmp/ipykernel_27677/1432961727.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  grid_gdf["lon"] = grid_gdf.geometry.centroid.x


In [4]:
# Initialize an empty list to hold xarrays
xarrays_list = []

for i, time in enumerate(pd_datetime_list):
    if i % 500 == 0:
        print(i)
    no2_df = sensor_df_joined[sensor_df_joined['time']==time]
    time_df = pd.merge(grid_gdf, no2_df, on=['lat', 'lon'], how='left')
    time_df['time'] = time_df['time'].fillna(time)
    
    # Pivot the data into a 2D array
    time_df_pivot = time_df.pivot(index="lat", columns="lon", values="no2")

    # Convert to xarray
    da = xr.DataArray(
        time_df_pivot.values,
        coords={
            "lat": time_df_pivot.index.values,
            "lon": time_df_pivot.columns.values,
        },
        dims=["lat", "lon"],
        name="no2"
    )
    ds_expanded = da.expand_dims({'time': [time]})
    # Append the expanded xarray to the list
    xarrays_list.append(ds_expanded)

# Concatenate all the xarrays in the list along the 'time' dimension
final_ds = xr.concat(xarrays_list, dim='time')

final_ds.to_netcdf(f'../../data/sensor_data/full_gridded_sensoronly.nc', engine='h5netcdf')

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000


ModuleNotFoundError: No module named 'h5netcdf'

In [ ]:
final_ds.to_netcdf(f'../../data/sensor_data/full_gridded_sensoronly.nc')

# Sensor and Mobile Data

In [ ]:
sensor_df_joined=pd.read_csv('../../data/sensor_data/final_sensor_withgrid.csv')
mobile_df = pd.read_csv('../../data/mobile_air_data/final_sensor_withgrid.csv')

In [ ]:
# # Initialize an empty list to hold xarrays
# xarrays_list = []

# for i, time in enumerate(pd_datetime_list):
#     print(i)
#     no2_df = final_df[final_df['time']==time]
#     time_df = pd.merge(grid_gdf, no2_df, on=['geometry', 'lat', 'lon'], how='left')
#     time_df['time'] = time_df['time'].fillna(time)
    
#     # Pivot the data into a 2D array
#     time_df_pivot = time_df.pivot(index="lat", columns="lon", values="weighted_no2")

#     # Convert to xarray
#     da = xr.DataArray(
#         time_df_pivot.values,
#         coords={
#             "lat": time_df_pivot.index.values,
#             "lon": time_df_pivot.columns.values,
#         },
#         dims=["lat", "lon"],
#         name="no2"
#     )
#     ds_expanded = da.expand_dims({'time': [time]})
#     # Append the expanded xarray to the list
#     xarrays_list.append(ds_expanded)

# # Concatenate all the xarrays in the list along the 'time' dimension
# final_ds = xr.concat(xarrays_list, dim='time')

In [ ]:
# final_ds.to_netcdf(f'../../data/mobile_air_data/full_gridded_mobile.nc', engine='h5netcdf')

In [ ]:
import matplotlib.pyplot as plt
# # # Define your latitude and longitude bounds
# # lat_min, lat_max = 28.6, 33.4  # Example latitude range
# # lon_min, lon_max = -98.9, -88.3  # Example longitude range

# # Select one hour of temperature data (e.g., the first timestamp)
# hour_index = 1  # Change this to select a different hour if desired
# # temperature_data = nldas_month['temperature'].isel(time=hour_index)
# temperature_data = nldas_month['wind_u']

# Plot the data with switched axes
plt.figure(figsize=(10, 6))

# Plot with latitude on x-axis and longitude on y-axis
final_ds[175].plot(
    cmap="Reds",  # Colormap for temperature visualization
    cbar_kwargs={'label': 'Temperature (K)'}  # Add color bar label
)

# Update axis labels
plt.xlabel("Latitude")
plt.ylabel("Longitude")

plt.show()